In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import os
import gc
import warnings
warnings.filterwarnings('ignore')

from tslearn.generators import random_walks
from tslearn.clustering import TimeSeriesKMeans

import tensorflow as tf
import keras
from keras import layers, models, optimizers
from keras.regularizers import L1L2
from keras.callbacks import EarlyStopping, LearningRateScheduler
import keras.backend as K
from tensorflow.keras.losses import Huber

tf.config.experimental.set_visible_devices([], 'GPU')

import xgboost as xgb
import lightgbm as lgb
import catboost as cat
from bayes_opt import BayesianOptimization

from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import log_loss, accuracy_score, f1_score, r2_score, mean_squared_error, silhouette_score
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression, ElasticNet, Lasso
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture
from sklearn import metrics

from pystacknet.pystacknet import StackNetClassifier, StackNetRegressor

import impyute
import datawig


Using TensorFlow backend.


In [2]:
tr = pd.read_csv('./data/train.csv')
te = pd.read_csv('./data/test.csv')
target = tr['count']
# m, s = np.mean(target), np.std(target)
# target = (target - np.min(target))/(np.max(target) - np.min(target))
# t = (target - m)/s

sub = pd.read_csv('./data/submission.csv')

In [ ]:
tr

In [3]:
temp = impyute.mice(tr)
temp.columns = tr.columns
tr = temp.copy()

temp = impyute.mice(te)
temp.columns = te.columns
te = temp.copy()

In [4]:
categorical = [
    'mise', 'chomise', 'ozone'
]

In [5]:
def mise(a):
    if a <= 30:
        return 0
    elif a <= 80:
        return 1
    elif a <= 150:
        return 2
    else:
        return 3
    
def chomise(a):
    if a <= 15:
        return 0
    elif a <= 35:
        return 1
    elif a <= 75:
        return 2
    else:
        return 3
    
def ozone(a):
    if a <= 0.03:
        return 0
    elif a <= 0.09:
        return 1
    elif a <= 0.15:
        return 2
    else:
        return 3

In [ ]:
sns.distplot(target)

In [ ]:
plt.plot(tr.groupby(tr['hour']).mean()['count'])

In [6]:
down = [0, 1, 2, 3, 4, 9, 10, 11, 19, 20, 21, 22, 23]
up = [5, 6, 7, 8, 12, 13, 14, 15, 16, 17, 18]

In [7]:
tr['att'] = np.zeros(len(tr))
te['att'] = np.zeros(len(te))
for i in range(len(tr)):
    if tr['hour'][i] in down:
        tr['att'][i] = 1

for i in range(len(te)):
    if te['hour'][i] in down:
        te['att'][i] = 1

In [8]:
target = tr['count']
# target = np.log(target)

train_X = tr.drop(['id', 'count'], axis=1)
test_X = te.drop('id', axis=1)



In [9]:
train_X

,hour,hour_bef_temperature,hour_bef_precipitation,hour_bef_windspeed,hour_bef_humidity,hour_bef_visibility,hour_bef_ozone,hour_bef_pm10,hour_bef_pm2.5,att
0,20.0,16.3,1.0,1.5,89.0,576.0,0.027,76.0,33.0,1.0
1,13.0,20.1,0.0,1.4,48.0,916.0,0.042,73.0,40.0,0.0
2,6.0,13.9,0.0,0.7,79.0,1382.0,0.033,32.0,19.0,0.0
3,23.0,8.1,0.0,2.7,54.0,946.0,0.040,75.0,64.0,1.0
4,18.0,29.5,0.0,4.8,7.0,2000.0,0.057,27.0,11.0,0.0
...,...,...,...,...,...,...,...,...,...,...
1454,4.0,16.8,0.0,1.6,53.0,2000.0,0.031,37.0,27.0,1.0
1455,3.0,10.8,0.0,3.8,45.0,2000.0,0.039,34.0,19.0,1.0
1456,5.0,18.3,0.0,1.9,54.0,2000.0,0.009,30.0,21.0,0.0
1457,21.0,20.7,0.0,3.7,37.0,1395.0,0.082,71.0,36.0,1.0


In [10]:
train_X['idx'] = np.ones(len(train_X))
test_X['idx'] = np.zeros(len(test_X))

categorical = [
    'mise', 'chomise', 'ozone'
]

merge = pd.concat([train_X, test_X], ignore_index=True)

# for c in merge.columns[0:-1]:
#     merge[c].fillna(merge[c].mode()[0], inplace=True)

# for c in merge.columns:
#     if np.sum(merge[c].isna()) > 0: 
#         imputer = datawig.SimpleImputer(
#             input_columns=merge.columns, # column(s) containing information about the column we want to impute
#             output_column=c, # the column we'd like to impute values for
#             output_path='./imp'
#             )
        
#         imputer.fit(train_df=merge)
        
#         imputed = imputer.predict(merge)
#         merge[c] = imputed.iloc[:,-1]
        
merge['mise'] = merge['hour_bef_pm10'].map(mise)
merge['chomise'] = merge['hour_bef_pm2.5'].map(chomise)
merge['ozone'] = merge['hour_bef_ozone'].map(mise)
        
# merge = merge.drop('hour', axis=1)

# dum = pd.get_dummies(merge['hour'], prefix='hour')
# merge = pd.concat([merge, dum], axis=1)
# merge = merge.drop('hour', axis=1)
    
# merge['hour_bef_visibility'] = np.log(merge['hour_bef_visibility'])
# merge['hour_bef_temperature'] = np.log(merge['hour_bef_temperature'])
# merge['hour_bef_pm10'] = np.log(merge['hour_bef_pm10'])
# merge['hour_bef_pm2.5'] = np.log(merge['hour_bef_pm2.5'])
# merge['hour_bef_windspeed'] = np.log(merge['hour_bef_windspeed'])

# merge['hour_bef_humidity'] /= 100

# train_X = merge[merge['idx'] == 1].drop('idx', axis=1)
# test_X = merge[merge['idx'] == 0].drop('idx', axis=1)

In [11]:
for c in merge.columns:
    if 'bef' in c and c != 'hour_bef_precipitation':
        print(c)
        merge[c] = (merge[c] - np.min(merge[c])) / (np.max(merge[c]) - np.min(merge[c]))

hour_bef_temperature
hour_bef_windspeed
hour_bef_humidity
hour_bef_visibility
hour_bef_ozone
hour_bef_pm10
hour_bef_pm2.5


In [ ]:
merge

In [12]:
hour = pd.get_dummies(merge['hour'], prefix='hour')
tr_hour = hour.iloc[0:len(tr), :]
te_hour = hour.iloc[len(tr):, :]

t_merge = merge.drop(['hour', 'idx'], axis=1)

In [ ]:
ans = []
sils = []
for i in range(3, 30):
    km = GaussianMixture(n_components=i, random_state=42)

    res = km.fit(t_merge)
    
#     ans.append(res.inertia_)
    sils.append(silhouette_score(t_merge, res.predict(t_merge)))

plt.plot(range(3,30), ans, '-o')
plt.show()
plt.plot(range(3,30), sils, '-o')
plt.show()

In [13]:
km = GaussianMixture(n_components=19, random_state=42).fit(t_merge)

In [14]:
cl = km.predict(t_merge)

cls = pd.get_dummies(cl, prefix='cl')
tr_cl = cls.iloc[0:len(tr), :]
te_cl = cls.iloc[len(tr):, :]

In [ ]:
### diff

In [15]:
# dum = pd.get_dummies(merge['cl'], prefix='cl')
# merge = pd.concat([merge, dum], axis=1)
# merge = merge.drop('cl', axis=1)

# for c in merge.columns:
#     if 'bef' in c:
#         merge[c] = (merge[c] - np.min(merge[c]))/(np.max(merge[c]) - np.min(merge[c]))

train_X = merge[merge['idx'] == 1].drop(['idx', 'att'], axis=1)
test_X = merge[merge['idx'] == 0].drop(['idx', 'att'], axis=1)

In [16]:
train_X

,hour,hour_bef_temperature,hour_bef_precipitation,hour_bef_windspeed,hour_bef_humidity,hour_bef_visibility,hour_bef_ozone,hour_bef_pm10,hour_bef_pm2.5,mise,chomise,ozone
0,20.0,0.429967,1.0,0.1875,0.881720,0.259105,0.177778,0.257692,0.313253,1,1,0
1,13.0,0.553746,0.0,0.1750,0.440860,0.436004,0.288889,0.246154,0.397590,1,2,0
2,6.0,0.351792,0.0,0.0875,0.774194,0.678460,0.222222,0.088462,0.144578,1,1,0
3,23.0,0.162866,0.0,0.3375,0.505376,0.451613,0.274074,0.253846,0.686747,1,2,0
4,18.0,0.859935,0.0,0.6000,0.000000,1.000000,0.400000,0.069231,0.048193,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
1454,4.0,0.446254,0.0,0.2000,0.494624,1.000000,0.207407,0.107692,0.240964,1,1,0
1455,3.0,0.250814,0.0,0.4750,0.408602,1.000000,0.266667,0.096154,0.144578,1,1,0
1456,5.0,0.495114,0.0,0.2375,0.505376,1.000000,0.044444,0.080769,0.168675,0,1,0
1457,21.0,0.573290,0.0,0.4625,0.322581,0.685224,0.585185,0.238462,0.349398,1,2,0


In [17]:
tr_X, te_X, tr_y, te_y = train_test_split(train_X, target, test_size=0.2, random_state=42, shuffle=True)


In [18]:
def LGB_bayesian(
    #learning_rate,
    num_leaves, 
    bagging_fraction,
    feature_fraction,
    min_child_weight, 
    min_data_in_leaf,
    max_depth,
    reg_alpha,
    reg_lambda
     ):
    # LightGBM expects next three parameters need to be integer. 
    num_leaves = int(num_leaves)
    min_data_in_leaf = int(min_data_in_leaf)
    max_depth = int(max_depth)

    assert type(num_leaves) == int
    assert type(min_data_in_leaf) == int
    assert type(max_depth) == int
    

    params = {
              'num_leaves': num_leaves, 
              'min_data_in_leaf': min_data_in_leaf,
              'min_child_weight': min_child_weight,
              'bagging_fraction' : bagging_fraction,
              'feature_fraction' : feature_fraction,
#               'learning_rate' : 0.03,
              'max_depth': max_depth,
              'reg_alpha': reg_alpha,
              'reg_lambda': reg_lambda,
              'objective': 'regression',
              'save_binary': True,
              'seed': 12,
              'feature_fraction_seed': 12,
              'bagging_seed': 12,
              'drop_seed': 12,
              'data_random_seed': 12,
              'boosting': 'gbdt', ## some get better result using 'dart'
              'verbose': 1,
              'is_unbalance': False,
              'boost_from_average': True,
              'metric':'rmse'}    
    
    ## set clf options
    clf = lgb.LGBMRegressor(**params).fit(tr_X, tr_y, early_stopping_rounds=100,eval_set=[(te_X, te_y)], eval_metric='rmse', verbose=0)
    
    score = -mean_squared_error(te_y, clf.predict(te_X))

    return score


In [19]:
bounds_LGB = {
    'num_leaves': (10, 500), 
    'min_data_in_leaf': (0, 150),
    'bagging_fraction' : (0.1, 0.9),
    'feature_fraction' : (0.1, 0.9),
#     'learning_rate': (0.01, 0.3),
    'min_child_weight': (0.001, 3),   
    'reg_alpha': (0.1, 3), 
    'reg_lambda': (0.1, 3),
    'max_depth':(3, 19),
}

In [20]:
optimizer = BayesianOptimization(LGB_bayesian, bounds_LGB, random_state=12)


In [21]:
init_points = 15
n_iter = 15

optimizer.maximize(init_points=init_points, n_iter=n_iter)


|   iter    |  target   | baggin... | featur... | max_depth | min_ch... | min_da... | num_le... | reg_alpha | reg_la... |
-------------------------------------------------------------------------------------------------------------------------
|  1        | -1.353e+0 |  0.2233   |  0.692    |  7.213    |  1.602    |  2.186    |  460.2    |  2.712    |  0.1969   |
|  2        | -1.836e+0 |  0.8656   |  0.2098   |  7.541    |  1.819    |  141.6    |  427.8    |  0.1066   |  1.612    |
|  3        | -1.627e+0 |  0.5416   |  0.4883   |  15.29    |  0.483    |  114.7    |  20.2     |  0.4921   |  0.4372   |
|  4        | -1.396e+0 |  0.3479   |  0.6372   |  10.54    |  2.449    |  43.44    |  369.2    |  2.138    |  1.05     |
|  5        | -1.656e+0 |  0.3677   |  0.8824   |  12.99    |  2.851    |  115.1    |  414.3    |  1.279    |  1.409    |
|  6        | -1.529e+0 |  0.4205   |  0.8961   |  5.841    |  2.888    |  62.89    |  217.8    |  1.443    |  1.184    |
|  7        | -1.656e+0 

In [22]:
param_lgb = {
        'min_data_in_leaf': int(optimizer.max['params']['min_data_in_leaf']), 
        'num_leaves': int(optimizer.max['params']['num_leaves']), 
        #'learning_rate': LGB_BO.max['params']['learning_rate'],
        'min_child_weight': optimizer.max['params']['min_child_weight'],
        'bagging_fraction': optimizer.max['params']['bagging_fraction'], 
        'feature_fraction': optimizer.max['params']['feature_fraction'],
        'reg_lambda': optimizer.max['params']['reg_lambda'],
        'reg_alpha': optimizer.max['params']['reg_alpha'],
        'max_depth': int(optimizer.max['params']['max_depth']), 
        'objective': 'regression',
        'save_binary': True,
        'seed': 12,
        'feature_fraction_seed': 12,
        'bagging_seed': 12,
        'drop_seed': 12,
        'data_random_seed': 12,
        'boosting_type': 'gbdt',  # also consider 'dart'
        'verbose': 1,
        'is_unbalance': False,
        'boost_from_average': True,
        'metric':'rmse'
    }

params = param_lgb.copy()

In [ ]:
params

In [23]:
lgb_clf = lgb.LGBMRegressor(**params, early_stoppong_rounds = 50)
lgb_clf.fit(train_X, target)

LGBMRegressor(bagging_fraction=0.2969023859017197, bagging_seed=12,
              boost_from_average=True, boosting_type='gbdt', class_weight=None,
              colsample_bytree=1.0, data_random_seed=12, drop_seed=12,
              early_stoppong_rounds=50, feature_fraction=0.84125315554184,
              feature_fraction_seed=12, importance_type='split',
              is_unbalance=False, learning_rate=0.1, max_depth=6, metric='rmse',
              min_child_samples=20, min_child_weight=2.8175820976128976,
              min_data_in_leaf=10, min_split_gain=0.0, n_estimators=100,
              n_jobs=-1, num_leaves=302, objective='regression',
              random_state=None, reg_alpha=1.1591997532781917,
              reg_lambda=1.8378753119406754, save_binary=True, seed=12,
              silent=True, ...)

In [24]:
sub['count'] = lgb_clf.predict(test_X)
print(sub)

       id       count
0       0  112.959205
1       1  178.005265
2       2   68.437301
3       4   55.320903
4       5   75.978250
..    ...         ...
710  2148   68.972114
711  2149   69.888791
712  2165  117.730948
713  2166  122.843367
714  2177  190.681399

[715 rows x 2 columns]


In [25]:
sub.to_csv('./sub/pre/lgbm12.csv', index=False)

In [26]:
# parmas for xgboost
params_fx = {'min_data_in_leaf': params['min_data_in_leaf'],
             'num_leaves': params['num_leaves'],
             'min_child_weight': params['min_child_weight'],
             'bagging_fraction': params['bagging_fraction'],
             'feature_fraction': params['feature_fraction'],
             'reg_lambda': params['reg_lambda'],
             'reg_alpha': params['reg_alpha'],
             'max_depth': params['max_depth'],
#              'obective': 'reg:squarederror',
             'eval_metric': 'rmse'
             
}

In [27]:
xgb_clf = xgb.XGBRegressor(
            **params_fx,
            early_stoppong_rounds = 50         
    ).fit(train_X.values, target)

[10:24:44] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [28]:
sub['count'] = xgb_clf.predict(test_X.values)
print(sub)

       id       count
0       0  108.160751
1       1  198.448853
2       2   58.187916
3       4   42.352539
4       5   33.390259
..    ...         ...
710  2148   68.807144
711  2149   67.608841
712  2165  125.937126
713  2166  150.527924
714  2177  173.627350

[715 rows x 2 columns]


In [29]:
sub.to_csv('./sub/pre/xgb8.csv', index=False)

In [30]:
cat_clf = cat.CatBoostRegressor(random_seed=42, n_estimators=2000, early_stopping_rounds=50).fit(train_X, target)

Learning rate set to 0.02465
0:	learn: 81.5670598	total: 64.9ms	remaining: 2m 9s
1:	learn: 80.3869847	total: 66.3ms	remaining: 1m 6s
2:	learn: 79.1593664	total: 67.5ms	remaining: 44.9s
3:	learn: 78.0413033	total: 69ms	remaining: 34.4s
4:	learn: 76.8971335	total: 70.4ms	remaining: 28.1s
5:	learn: 75.8576879	total: 72ms	remaining: 23.9s
6:	learn: 74.8104950	total: 73.5ms	remaining: 20.9s
7:	learn: 74.0000768	total: 75.7ms	remaining: 18.8s
8:	learn: 73.0410460	total: 77.3ms	remaining: 17.1s
9:	learn: 72.1826818	total: 79.2ms	remaining: 15.8s
10:	learn: 71.2661677	total: 80.8ms	remaining: 14.6s
11:	learn: 70.4732867	total: 82.2ms	remaining: 13.6s
12:	learn: 69.5964869	total: 84.1ms	remaining: 12.9s
13:	learn: 68.7098429	total: 85.9ms	remaining: 12.2s
14:	learn: 67.8621856	total: 87.6ms	remaining: 11.6s
15:	learn: 67.0914886	total: 89.1ms	remaining: 11s
16:	learn: 66.3120438	total: 90.4ms	remaining: 10.5s
17:	learn: 65.5999860	total: 91.7ms	remaining: 10.1s
18:	learn: 64.8826551	total: 93.4

173:	learn: 36.9937093	total: 309ms	remaining: 3.24s
174:	learn: 36.9740535	total: 310ms	remaining: 3.23s
175:	learn: 36.9224275	total: 312ms	remaining: 3.23s
176:	learn: 36.8833454	total: 313ms	remaining: 3.22s
177:	learn: 36.8342223	total: 314ms	remaining: 3.21s
178:	learn: 36.7819571	total: 315ms	remaining: 3.21s
179:	learn: 36.7469888	total: 317ms	remaining: 3.2s
180:	learn: 36.7173758	total: 318ms	remaining: 3.19s
181:	learn: 36.6616922	total: 319ms	remaining: 3.19s
182:	learn: 36.6277972	total: 320ms	remaining: 3.18s
183:	learn: 36.5991584	total: 322ms	remaining: 3.17s
184:	learn: 36.5698983	total: 323ms	remaining: 3.17s
185:	learn: 36.5318330	total: 324ms	remaining: 3.16s
186:	learn: 36.5064478	total: 326ms	remaining: 3.15s
187:	learn: 36.4729722	total: 327ms	remaining: 3.15s
188:	learn: 36.4530047	total: 328ms	remaining: 3.14s
189:	learn: 36.4038327	total: 329ms	remaining: 3.14s
190:	learn: 36.3741847	total: 331ms	remaining: 3.13s
191:	learn: 36.3359181	total: 332ms	remaining: 

417:	learn: 30.6153786	total: 626ms	remaining: 2.37s
418:	learn: 30.5884687	total: 628ms	remaining: 2.37s
419:	learn: 30.5580905	total: 629ms	remaining: 2.37s
420:	learn: 30.5350466	total: 630ms	remaining: 2.36s
421:	learn: 30.5290283	total: 632ms	remaining: 2.36s
422:	learn: 30.5227785	total: 633ms	remaining: 2.36s
423:	learn: 30.5186321	total: 634ms	remaining: 2.36s
424:	learn: 30.4916087	total: 635ms	remaining: 2.35s
425:	learn: 30.4634543	total: 636ms	remaining: 2.35s
426:	learn: 30.4406171	total: 638ms	remaining: 2.35s
427:	learn: 30.4242693	total: 639ms	remaining: 2.35s
428:	learn: 30.4076794	total: 641ms	remaining: 2.35s
429:	learn: 30.3910702	total: 642ms	remaining: 2.34s
430:	learn: 30.3741532	total: 643ms	remaining: 2.34s
431:	learn: 30.3537561	total: 644ms	remaining: 2.34s
432:	learn: 30.3271840	total: 646ms	remaining: 2.34s
433:	learn: 30.2966475	total: 647ms	remaining: 2.33s
434:	learn: 30.2798270	total: 648ms	remaining: 2.33s
435:	learn: 30.2759297	total: 649ms	remaining:

648:	learn: 26.4275074	total: 930ms	remaining: 1.94s
649:	learn: 26.4131584	total: 932ms	remaining: 1.93s
650:	learn: 26.3775861	total: 933ms	remaining: 1.93s
651:	learn: 26.3683487	total: 934ms	remaining: 1.93s
652:	learn: 26.3516631	total: 935ms	remaining: 1.93s
653:	learn: 26.3271455	total: 937ms	remaining: 1.93s
654:	learn: 26.3057488	total: 938ms	remaining: 1.93s
655:	learn: 26.2802326	total: 939ms	remaining: 1.92s
656:	learn: 26.2648338	total: 940ms	remaining: 1.92s
657:	learn: 26.2506441	total: 942ms	remaining: 1.92s
658:	learn: 26.2351892	total: 943ms	remaining: 1.92s
659:	learn: 26.2142384	total: 944ms	remaining: 1.92s
660:	learn: 26.2023731	total: 946ms	remaining: 1.92s
661:	learn: 26.1840143	total: 947ms	remaining: 1.91s
662:	learn: 26.1791947	total: 949ms	remaining: 1.91s
663:	learn: 26.1522241	total: 950ms	remaining: 1.91s
664:	learn: 26.1311522	total: 951ms	remaining: 1.91s
665:	learn: 26.1194511	total: 952ms	remaining: 1.91s
666:	learn: 26.1081296	total: 954ms	remaining:

897:	learn: 22.5054447	total: 1.25s	remaining: 1.53s
898:	learn: 22.4802822	total: 1.25s	remaining: 1.53s
899:	learn: 22.4766905	total: 1.25s	remaining: 1.53s
900:	learn: 22.4738428	total: 1.25s	remaining: 1.53s
901:	learn: 22.4604783	total: 1.25s	remaining: 1.52s
902:	learn: 22.4477960	total: 1.25s	remaining: 1.52s
903:	learn: 22.4453964	total: 1.25s	remaining: 1.52s
904:	learn: 22.4297145	total: 1.26s	remaining: 1.52s
905:	learn: 22.4176648	total: 1.26s	remaining: 1.52s
906:	learn: 22.3965364	total: 1.26s	remaining: 1.52s
907:	learn: 22.3828750	total: 1.26s	remaining: 1.52s
908:	learn: 22.3656499	total: 1.26s	remaining: 1.51s
909:	learn: 22.3593388	total: 1.26s	remaining: 1.51s
910:	learn: 22.3491322	total: 1.26s	remaining: 1.51s
911:	learn: 22.3354804	total: 1.27s	remaining: 1.51s
912:	learn: 22.3210976	total: 1.27s	remaining: 1.51s
913:	learn: 22.3096330	total: 1.27s	remaining: 1.51s
914:	learn: 22.3021457	total: 1.27s	remaining: 1.5s
915:	learn: 22.2913147	total: 1.27s	remaining: 

1136:	learn: 19.7187098	total: 1.56s	remaining: 1.19s
1137:	learn: 19.7060307	total: 1.56s	remaining: 1.19s
1138:	learn: 19.6993873	total: 1.57s	remaining: 1.18s
1139:	learn: 19.6957330	total: 1.57s	remaining: 1.18s
1140:	learn: 19.6916155	total: 1.57s	remaining: 1.18s
1141:	learn: 19.6830394	total: 1.57s	remaining: 1.18s
1142:	learn: 19.6778227	total: 1.57s	remaining: 1.18s
1143:	learn: 19.6688152	total: 1.57s	remaining: 1.18s
1144:	learn: 19.6553525	total: 1.57s	remaining: 1.18s
1145:	learn: 19.6495848	total: 1.58s	remaining: 1.17s
1146:	learn: 19.6370104	total: 1.58s	remaining: 1.17s
1147:	learn: 19.6302363	total: 1.58s	remaining: 1.17s
1148:	learn: 19.6207293	total: 1.58s	remaining: 1.17s
1149:	learn: 19.6077000	total: 1.58s	remaining: 1.17s
1150:	learn: 19.5990321	total: 1.58s	remaining: 1.17s
1151:	learn: 19.5900643	total: 1.58s	remaining: 1.17s
1152:	learn: 19.5829636	total: 1.58s	remaining: 1.16s
1153:	learn: 19.5774870	total: 1.59s	remaining: 1.16s
1154:	learn: 19.5668334	tota

1364:	learn: 17.5725458	total: 1.87s	remaining: 871ms
1365:	learn: 17.5662366	total: 1.87s	remaining: 870ms
1366:	learn: 17.5555704	total: 1.87s	remaining: 868ms
1367:	learn: 17.5465841	total: 1.88s	remaining: 867ms
1368:	learn: 17.5411086	total: 1.88s	remaining: 865ms
1369:	learn: 17.5285744	total: 1.88s	remaining: 864ms
1370:	learn: 17.5200851	total: 1.88s	remaining: 863ms
1371:	learn: 17.5118531	total: 1.88s	remaining: 861ms
1372:	learn: 17.5073001	total: 1.88s	remaining: 860ms
1373:	learn: 17.4963245	total: 1.88s	remaining: 859ms
1374:	learn: 17.4813036	total: 1.89s	remaining: 857ms
1375:	learn: 17.4729808	total: 1.89s	remaining: 856ms
1376:	learn: 17.4593214	total: 1.89s	remaining: 854ms
1377:	learn: 17.4488091	total: 1.89s	remaining: 853ms
1378:	learn: 17.4394717	total: 1.89s	remaining: 852ms
1379:	learn: 17.4312831	total: 1.89s	remaining: 850ms
1380:	learn: 17.4178266	total: 1.89s	remaining: 849ms
1381:	learn: 17.4071883	total: 1.9s	remaining: 848ms
1382:	learn: 17.4003005	total

1586:	learn: 15.8827946	total: 2.17s	remaining: 566ms
1587:	learn: 15.8821869	total: 2.18s	remaining: 565ms
1588:	learn: 15.8766750	total: 2.18s	remaining: 563ms
1589:	learn: 15.8677939	total: 2.18s	remaining: 562ms
1590:	learn: 15.8661801	total: 2.18s	remaining: 561ms
1591:	learn: 15.8571285	total: 2.18s	remaining: 559ms
1592:	learn: 15.8460495	total: 2.18s	remaining: 558ms
1593:	learn: 15.8317168	total: 2.18s	remaining: 556ms
1594:	learn: 15.8248173	total: 2.19s	remaining: 555ms
1595:	learn: 15.8209189	total: 2.19s	remaining: 554ms
1596:	learn: 15.8126010	total: 2.19s	remaining: 552ms
1597:	learn: 15.8030574	total: 2.19s	remaining: 551ms
1598:	learn: 15.7888934	total: 2.19s	remaining: 550ms
1599:	learn: 15.7810538	total: 2.19s	remaining: 548ms
1600:	learn: 15.7735422	total: 2.19s	remaining: 547ms
1601:	learn: 15.7616777	total: 2.19s	remaining: 545ms
1602:	learn: 15.7604878	total: 2.2s	remaining: 544ms
1603:	learn: 15.7595120	total: 2.2s	remaining: 543ms
1604:	learn: 15.7511704	total:

1828:	learn: 14.3391588	total: 2.49s	remaining: 233ms
1829:	learn: 14.3343464	total: 2.49s	remaining: 232ms
1830:	learn: 14.3231655	total: 2.5s	remaining: 230ms
1831:	learn: 14.3183948	total: 2.5s	remaining: 229ms
1832:	learn: 14.3148349	total: 2.5s	remaining: 228ms
1833:	learn: 14.3140433	total: 2.5s	remaining: 226ms
1834:	learn: 14.3059166	total: 2.5s	remaining: 225ms
1835:	learn: 14.3020339	total: 2.5s	remaining: 224ms
1836:	learn: 14.2959979	total: 2.5s	remaining: 222ms
1837:	learn: 14.2878369	total: 2.5s	remaining: 221ms
1838:	learn: 14.2858059	total: 2.51s	remaining: 219ms
1839:	learn: 14.2826341	total: 2.51s	remaining: 218ms
1840:	learn: 14.2741645	total: 2.51s	remaining: 217ms
1841:	learn: 14.2688131	total: 2.51s	remaining: 215ms
1842:	learn: 14.2659188	total: 2.51s	remaining: 214ms
1843:	learn: 14.2652572	total: 2.51s	remaining: 213ms
1844:	learn: 14.2579132	total: 2.51s	remaining: 211ms
1845:	learn: 14.2517912	total: 2.52s	remaining: 210ms
1846:	learn: 14.2437336	total: 2.52s

In [31]:
sub['count'] = cat_clf.predict(test_X.values)
print(sub)

       id       count
0       0  115.974796
1       1  214.136491
2       2   41.770848
3       4   65.175992
4       5   57.682344
..    ...         ...
710  2148   66.063924
711  2149   65.080200
712  2165  111.439199
713  2166  138.711589
714  2177  201.815166

[715 rows x 2 columns]


In [32]:
sub.to_csv('./sub/pre/cat5.csv', index=False)

In [33]:
# lgb_clf = lgb.LGBMRegressor(**params, early_stoppong_rounds = 100)

# xgb_clf = xgb.XGBRegressor(
#             **params_fx,
#             early_stoppong_rounds = 100         
#     )

# cat_clf = cat.CatBoostRegressor(random_seed=42, n_estimators=3000, early_stopping_rounds=50)

rf1 = RandomForestRegressor(n_estimators=200,
                                max_depth=13, 
                                max_features='sqrt', 
                                random_state=21)

rf2 = RandomForestRegressor(n_estimators=150,
                                max_depth=9, 
                                max_features='sqrt', 
                                random_state=12)

pca = PCA(9, random_state=42)

lr = LinearRegression()
els = ElasticNet(random_state=42)
las = Lasso(random_state=21)

In [34]:
models = [[lgb_clf, xgb_clf, cat_clf, pca], 
          [rf1]]

In [35]:
model = StackNetRegressor(models, 
                           metric="rmse", 
                           folds=4,
                           restacking=False,
                           random_state=42,
                           n_jobs=-1, 
                           verbose=1)

model.fit(train_X.values, target)

====================== Start of Level 0 ======================
Input Dimensionality 12 at Level 0 
4 models included in Level 0 
Fold 1/4 , model 0 , rmse===35.375552 
Fold 1/4 , model 1 , rmse===35.236288 
Fold 1/4 , model 2 , rmse===35.232695 
=========== end of fold 1 in level 0 ===========
Fold 2/4 , model 0 , rmse===40.200837 
Fold 2/4 , model 1 , rmse===38.983266 
Fold 2/4 , model 2 , rmse===37.262065 
=========== end of fold 2 in level 0 ===========
Fold 3/4 , model 0 , rmse===39.239290 
Fold 3/4 , model 1 , rmse===39.181571 
Fold 3/4 , model 2 , rmse===37.571293 
=========== end of fold 3 in level 0 ===========
Fold 4/4 , model 0 , rmse===38.281710 
Fold 4/4 , model 1 , rmse===37.246254 
Fold 4/4 , model 2 , rmse===35.389164 
=========== end of fold 4 in level 0 ===========
Output dimensionality of level 0 is 12 
====================== End of Level 0 ======================
 level 0 lasted 42.443017 seconds 
====================== Start of Level 1 ======================
Input Di

In [36]:
sub['count'] = model.predict(test_X.values)
print(sub)

====================== Start of Level 0 ======================
1 estimators included in Level 0 
====================== Start of Level 1 ======================
1 estimators included in Level 1 
       id       count
0       0  117.622388
1       1  219.484277
2       2   73.639982
3       4   56.997338
4       5   74.202702
..    ...         ...
710  2148   63.739009
711  2149   59.704261
712  2165  121.479439
713  2166  125.590771
714  2177  187.878675

[715 rows x 2 columns]


In [37]:
sub.to_csv('./sub/pre/stk16.csv', index=False)

In [38]:
k1 = model.predict_up_to(train_X.values)
k2 = model.predict_up_to(test_X.values)

new_tr_X = k1[0]
new_te_X = k2[0]

====================== Start of Level 0 ======================
1 estimators included in Level 0 
(1459, 12)
====================== Start of Level 1 ======================
1 estimators included in Level 1 
(1459, 1)
====================== Start of Level 0 ======================
1 estimators included in Level 0 
(715, 12)
====================== Start of Level 1 ======================
1 estimators included in Level 1 
(715, 1)


In [39]:
temp_merge = pd.concat([train_X, test_X])
weather = pd.get_dummies(temp_merge[['mise', 'chomise', 'ozone']], prefix='w')
tr_w = weather.iloc[0:len(tr), :]
te_w = weather.iloc[len(tr):, :]

train_X = train_X.drop(['hour', 'mise', 'chomise', 'ozone'], axis=1)
test_X = test_X.drop(['hour', 'mise', 'chomise', 'ozone'], axis=1)

In [40]:
train_X

,hour_bef_temperature,hour_bef_precipitation,hour_bef_windspeed,hour_bef_humidity,hour_bef_visibility,hour_bef_ozone,hour_bef_pm10,hour_bef_pm2.5
0,0.429967,1.0,0.1875,0.881720,0.259105,0.177778,0.257692,0.313253
1,0.553746,0.0,0.1750,0.440860,0.436004,0.288889,0.246154,0.397590
2,0.351792,0.0,0.0875,0.774194,0.678460,0.222222,0.088462,0.144578
3,0.162866,0.0,0.3375,0.505376,0.451613,0.274074,0.253846,0.686747
4,0.859935,0.0,0.6000,0.000000,1.000000,0.400000,0.069231,0.048193
...,...,...,...,...,...,...,...,...
1454,0.446254,0.0,0.2000,0.494624,1.000000,0.207407,0.107692,0.240964
1455,0.250814,0.0,0.4750,0.408602,1.000000,0.266667,0.096154,0.144578
1456,0.495114,0.0,0.2375,0.505376,1.000000,0.044444,0.080769,0.168675
1457,0.573290,0.0,0.4625,0.322581,0.685224,0.585185,0.238462,0.349398


In [ ]:
dum = pd.get_dummies(merge['hour'], prefix='hour')
merge = pd.concat([merge, dum], axis=1)
merge = merge.drop('hour', axis=1)

dum = pd.get_dummies(merge['cl'], prefix='cl')
merge = pd.concat([merge, dum], axis=1)
merge = merge.drop('cl', axis=1)

In [41]:
def DenseBlock(n, activation='relu'):
    def f(x):
        x = layers.Dense(n, kernel_initializer='he_normal')(x) #  kernel_regularizer=L1L2(l2=0.0001)
#         x = layers.BatchNormalization(momentum=0.8)(x)
        x = layers.advanced_activations.LeakyReLU(0.3)(x)
        
        return x
    return f

es = keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)

In [42]:
# inputs = [train_X.values, tr_hour, tr_cl, tr_w]
inputs = [new_tr_X, tr_hour, tr_cl, tr_w]
# te_inputs = [test_X.values, te_hour, te_cl, te_w]
te_inputs = [new_te_X, te_hour, te_cl, te_w]


In [ ]:
inputs = [new_tr_X, tr_hour, tr_w]
te_inputs = [new_te_X, te_hour, te_w]

In [50]:
K.clear_session()
h_dim = [32, 16]

inputs1 = layers.Input(shape=(inputs[0].shape[1], ))
inputs2 = layers.Input(shape=(inputs[1].shape[1], ))
inputs3 = layers.Input(shape=(inputs[2].shape[1], ))
inputs4 = layers.Input(shape=(inputs[3].shape[1], ))
# inputs4 = layers.Input(shape=(inputs[3].shape[1], ))


x1 = DenseBlock(h_dim[0])(inputs1)
x2 = DenseBlock(h_dim[0])(inputs2)
x3 = DenseBlock(h_dim[0])(inputs3)
x4 = DenseBlock(h_dim[0])(inputs4)

x1 = DenseBlock(h_dim[1])(x1)
x2 = DenseBlock(h_dim[1])(x2)
x3 = DenseBlock(h_dim[1])(x3)
x4 = DenseBlock(h_dim[1])(x4)


x = layers.Concatenate()([x1, x2, x3, x4])

x = DenseBlock(16)(x)

x = layers.Dense(1, activation='linear')(x)

nn = keras.models.Model([inputs1, inputs2, inputs3, inputs4], x)

nn.compile(optimizer = optimizers.Adam(lr=1e-4),
         loss = Huber(),
           metrics = ['mae']
)

In [51]:
# tf.keras.utils.plot_model(
#     nn,
#     show_shapes=False,
#     show_layer_names=True,
#     rankdir='TB'
# )

In [52]:
t = np.log(target)

In [53]:
nn.fit(inputs, t, 
     epochs=200,
     validation_split=0.3,
     callbacks=[es])

Train on 1021 samples, validate on 438 samples
Epoch 1/200
1021/1021 [==============================] - 0s 358us/step - loss: 5.7532 - mae: 6.2126 - val_loss: 1.7340 - val_mae: 2.1673
Epoch 2/200
1021/1021 [==============================] - 0s 71us/step - loss: 1.2618 - mae: 1.6788 - val_loss: 1.0577 - val_mae: 1.4536
Epoch 3/200
1021/1021 [==============================] - 0s 81us/step - loss: 1.1179 - mae: 1.5304 - val_loss: 1.0842 - val_mae: 1.4908
Epoch 4/200
1021/1021 [==============================] - 0s 72us/step - loss: 1.0984 - mae: 1.5058 - val_loss: 0.9961 - val_mae: 1.3895
Epoch 5/200
1021/1021 [==============================] - 0s 68us/step - loss: 1.0297 - mae: 1.4367 - val_loss: 0.9976 - val_mae: 1.3972
Epoch 6/200
1021/1021 [==============================] - 0s 78us/step - loss: 0.9836 - mae: 1.3900 - val_loss: 0.9187 - val_mae: 1.3084
Epoch 7/200
1021/1021 [==============================] - 0s 69us/step - loss: 0.9623 - mae: 1.3672 - val_loss: 0.8863 - val_mae: 1.2755


1021/1021 [==============================] - 0s 62us/step - loss: 0.0870 - mae: 0.3019 - val_loss: 0.0836 - val_mae: 0.2985
Epoch 61/200
1021/1021 [==============================] - 0s 59us/step - loss: 0.0883 - mae: 0.3013 - val_loss: 0.0871 - val_mae: 0.3092
Epoch 62/200
1021/1021 [==============================] - 0s 60us/step - loss: 0.0846 - mae: 0.2975 - val_loss: 0.0797 - val_mae: 0.2846
Epoch 63/200
1021/1021 [==============================] - 0s 60us/step - loss: 0.0827 - mae: 0.2900 - val_loss: 0.0935 - val_mae: 0.3265
Epoch 64/200
1021/1021 [==============================] - 0s 57us/step - loss: 0.0878 - mae: 0.3042 - val_loss: 0.0798 - val_mae: 0.2882
Epoch 65/200
1021/1021 [==============================] - 0s 70us/step - loss: 0.0800 - mae: 0.2855 - val_loss: 0.0795 - val_mae: 0.2871
Epoch 66/200
1021/1021 [==============================] - 0s 65us/step - loss: 0.0814 - mae: 0.2870 - val_loss: 0.0806 - val_mae: 0.2792
Epoch 67/200
1021/1021 [==============================

Epoch 120/200
1021/1021 [==============================] - 0s 67us/step - loss: 0.0638 - mae: 0.2497 - val_loss: 0.0692 - val_mae: 0.2543
Epoch 121/200
1021/1021 [==============================] - 0s 64us/step - loss: 0.0635 - mae: 0.2477 - val_loss: 0.0659 - val_mae: 0.2515
Epoch 122/200
1021/1021 [==============================] - 0s 63us/step - loss: 0.0608 - mae: 0.2406 - val_loss: 0.0670 - val_mae: 0.2486
Epoch 123/200
1021/1021 [==============================] - 0s 62us/step - loss: 0.0617 - mae: 0.2406 - val_loss: 0.0638 - val_mae: 0.2426
Epoch 124/200
1021/1021 [==============================] - 0s 66us/step - loss: 0.0644 - mae: 0.2508 - val_loss: 0.0635 - val_mae: 0.2409
Epoch 125/200
1021/1021 [==============================] - 0s 64us/step - loss: 0.0744 - mae: 0.2815 - val_loss: 0.0852 - val_mae: 0.3222
Epoch 126/200
1021/1021 [==============================] - 0s 72us/step - loss: 0.0787 - mae: 0.2951 - val_loss: 0.0795 - val_mae: 0.2823
Epoch 127/200
1021/1021 [=========

1021/1021 [==============================] - 0s 74us/step - loss: 0.0549 - mae: 0.2278 - val_loss: 0.0571 - val_mae: 0.2236
Epoch 180/200
1021/1021 [==============================] - 0s 71us/step - loss: 0.0635 - mae: 0.2534 - val_loss: 0.0633 - val_mae: 0.2444
Epoch 181/200
1021/1021 [==============================] - 0s 70us/step - loss: 0.0570 - mae: 0.2358 - val_loss: 0.0583 - val_mae: 0.2284
Epoch 182/200
1021/1021 [==============================] - 0s 70us/step - loss: 0.0557 - mae: 0.2301 - val_loss: 0.0630 - val_mae: 0.2537
Epoch 183/200
1021/1021 [==============================] - 0s 68us/step - loss: 0.0549 - mae: 0.2312 - val_loss: 0.0564 - val_mae: 0.2224
Epoch 184/200
1021/1021 [==============================] - 0s 68us/step - loss: 0.0562 - mae: 0.2323 - val_loss: 0.0668 - val_mae: 0.2698
Epoch 185/200
1021/1021 [==============================] - 0s 65us/step - loss: 0.0562 - mae: 0.2355 - val_loss: 0.0576 - val_mae: 0.2267
Epoch 186/200
1021/1021 [=======================

In [54]:
sub['count'] = np.exp(nn.predict(te_inputs))
print(sub)

       id       count
0       0   82.551483
1       1  216.502701
2       2   17.753199
3       4   29.626671
4       5    8.228092
..    ...         ...
710  2148   52.093311
711  2149   52.488117
712  2165  113.568932
713  2166  136.352844
714  2177  208.402512

[715 rows x 2 columns]


In [55]:
sub.to_csv('./sub/pre/new_nn8.csv', index=False)

In [ ]:
tr = pd.read_csv('./data/train.csv')
target = tr['count']
# m, s = np.mean(target), np.std(target)
mini, maxi = np.min(target), np.max(target)
# target = (target - np.min(target))/(np.max(target) - np.min(target))
# t = (target - m)/s


In [ ]:
target = np.log(target)
mini, maxi = np.min(target), np.max(target)
target = (target - np.min(target))/(np.max(target) - np.min(target))

In [ ]:
target

In [ ]:
pip install Impyute